<a href="https://colab.research.google.com/github/Xuanghdu/Genetic-Algorithm-Exercise-Black-Box-Attact/blob/master/Genetic_Algorithm_Exercise_Black_Box_Attact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf ./sample_data

In [0]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.3/deepspeech-0.7.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.3/deepspeech-0.7.3-models.scorer
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.3/audio-0.7.3.tar.gz
!tar xvf audio-0.7.3.tar.gz
!rm -f audio-0.7.3.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   639  100   639    0     0   2618      0 --:--:-- --:--:-- --:--:--  2608
100  180M  100  180M    0     0  25.9M      0  0:00:06  0:00:06 --:--:-- 32.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   641  100   641    0     0   2861      0 --:--:-- --:--:-- --:--:--  2861
100  909M  100  909M    0     0  27.7M      0  0:00:32  0:00:32 --:--:-- 27.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   629  100   629    0     0   2833      0 --:--:-- --:--:-- --:--:--  2833
100  193k  100  193k    0     0   207k      0 --:--:-- --:--:-- --:--:--  207k
._audio
audio/
audio/._2830-3980-0043.wav
audio/2830

In [0]:
!pip install deepspeech-gpu

     |████████████████████████████████| 19.2MB 1.2MB/s 


In [0]:
!deepspeech --model deepspeech-0.7.3-models.pbmm --scorer deepspeech-0.7.3-models.scorer --audio audio/8455-210777-0068.wav

2020-06-10 02:27:42.601352: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
Loading model from file deepspeech-0.7.3-models.pbmm
TensorFlow: v1.15.0-24-gceb46aa
DeepSpeech: v0.7.3-0-g8858494
2020-06-10 02:27:42.766507: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-10 02:27:42.767622: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-10 02:27:42.817296: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-10 02:27:42.817869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciB

In [0]:
import tensorflow as tf
import numpy as np
import sys
import scipy.io.wavfile as wav

tokens = " abcdefghijklmnopqrstuvwxyz'-"

class Genetic():
    def __init__(self, input_file, output_file, target_phrase):
        self.elite_size = 10
        self.pop_size = self.elite_size*self.elite_size*2
        self.crossover_size = self.elite_size*(self.elite_size-1)
        self.max_iters = 100
        self.noise_stdev = 50
        self.f, self.input_file = wav.read(input_file)
        self.input_file = self.input_file.astype(np.float32)
        self.pop = np.expand_dims(self.input_file, axis=0)
        self.pop = np.tile(self.pop, (self.pop_size, 1))
        self.output_file = output_file
        self.target_phrase = target_phrase
        self.target_enc = np.array([tokens.index(x) for x in target_phrase])

    def get_ctc_loss(self):
        # tf.nn.ctc_loss(labels, logits, label_length, logit_length)
        pass

    def first_mutate(self):
        self.pop[1:] += np.random.randn(*self.pop.shape)[1:] * self.noise_stdev

    def mutate(self):
        # self.pop[-self.pop_size/2:] += np.random.randn(*self.pop.shape)[-self.pop_size/2:] * noise_stdev
        pass

    def crossover(self, elite_pop):
        # update [-self.elite_size:] to elite_pop and crossover
        pass

    def attack(self):
        itr = 1
        best_text = ''
        self.first_mutate()
        # if log is not None:
        #     log.write('target phrase: ' + self.target_phrase + '\n')
        #     log.write('itr, corr, lev dist \n')
        while itr <= self.max_iters and best_text != self.target_phrase:
            pop_scores = self.get_ctc_loss()
            elite_ind = np.argsort(pop_scores)[-self.elite_size:]
            elite_pop, elite_pop_scores = self.pop[elite_ind], pop_scores[elite_ind]
            self.crossover(elite_pop, elite_pop_scores)
            self.mutate()
            itr += 1
            # input = the best of elite
            # best_text = tf.nn.ctc_greedy_decoder(inputs, sequence_length)
        return itr > self.max_iters

# input_file = sys.argv[1]
# target_phrase = sys.argv[2].lower()
input_file = 'sample_input.wav'
target_phrase = 'hello world'
output_file = input_file[:-4] + '_adv.wav'
# log_file = input_file[:-4] + '_log.txt'

print('target phrase: ', target_phrase)
print('source file: ', input_file)

g = Genetic(input_file, output_file, target_phrase)
# with open(log_file, 'w') as log:
    # success = g.attack(log=log)
success = g.attack()

if success:
    print('Success!') 
else:
    print('Failure!')
print('\nHere it is: ', output_file)

# with tf.device('/device:GPU:0'):